In [1]:
%reload_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
import os

import logging
logging.getLogger().setLevel(logging.INFO)

from alphabase.spectral_library.flat import SpecLibFlat
from alphabase.spectral_library.base import SpecLibBase
from alphabase.spectral_library.reader import SWATHLibraryReader
from alphadia.extraction.utils import reannotate_fragments
from alphabase.psm_reader import psm_reader_provider

INFO:root:


In [120]:
lib_location = '/Users/georgwallmann/Documents/data/alphadia_benchmarking/libraries/marvin_scp/MSfragger_library_mod_noLossType.tsv'
#insilico_lib_location = '/Users/georgwallmann/Documents/data/libraries/2022_111_brunner_2022_1ng_all_29_human_trypsin_diann_decoy.hdf'
output_location = '/Users/georgwallmann/Documents/data/alphadia_benchmarking/libraries/marvin_scp/MSfragger_library_mod_noLossType.hdf'

In [121]:
lib = pd.read_csv(lib_location, sep='\t')

In [77]:
lib = lib[lib['PeptideSequence'].str.contains('ALTHLEMR')]

In [122]:
lib

,PrecursorMz,ProductMz,Annotation,ProteinId,GeneName,PeptideSequence,ModifiedPeptideSequence,PrecursorCharge,LibraryIntensity,NormalizedRetentionTime,PrecursorIonMobility,FragmentType,FragmentCharge,FragmentSeriesNumber,AverageExperimentalRetentionTime
0,312.850644,274.187367,y2^1,Q9GZV5,WWTR1,LAGGAQHVR,(Dimethyl)LAGGAQHVR,3,10000.000000,5.802185,0.710272,y,1,2,293.134195
1,312.850644,411.246279,y3^1,Q9GZV5,WWTR1,LAGGAQHVR,(Dimethyl)LAGGAQHVR,3,4479.865772,5.802185,0.710272,y,1,3,293.134195
2,312.850644,539.304858,y4^1,Q9GZV5,WWTR1,LAGGAQHVR,(Dimethyl)LAGGAQHVR,3,2885.906040,5.802185,0.710272,y,1,4,293.134195
3,312.850644,610.341972,y5^1,Q9GZV5,WWTR1,LAGGAQHVR,(Dimethyl)LAGGAQHVR,3,1006.711409,5.802185,0.710272,y,1,5,293.134195
4,312.850644,724.384900,y7^1,Q9GZV5,WWTR1,LAGGAQHVR,(Dimethyl)LAGGAQHVR,3,7416.107383,5.802185,0.710272,y,1,7,293.134195
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
891218,1415.750903,743.393400,y7^1,P05387,RPLP2,LASVPAGGAVAVSAAPGSAAPAAGSAPAAAEEK,(Dimethyl)LASVPAGGAVAVSAAPGSAAPAAGSAPAAAEEK(Di...,2,7166.666667,46.251565,1.286647,y,1,7,933.127981
891219,1415.750903,851.498533,b10^1,P05387,RPLP2,LASVPAGGAVAVSAAPGSAAPAAGSAPAAAEEK,(Dimethyl)LASVPAGGAVAVSAAPGSAAPAAGSAPAAAEEK(Di...,2,10000.000000,46.251565,1.286647,b,1,10,933.127981
891220,1415.750903,922.535648,b11^1,P05387,RPLP2,LASVPAGGAVAVSAAPGSAAPAAGSAPAAAEEK,(Dimethyl)LASVPAGGAVAVSAAPGSAAPAAGSAPAAAEEK(Di...,2,3750.000000,46.251565,1.286647,b,1,11,933.127981
891221,1415.750903,1179.673205,b14^1,P05387,RPLP2,LASVPAGGAVAVSAAPGSAAPAAGSAPAAAEEK,(Dimethyl)LASVPAGGAVAVSAAPGSAAPAAGSAPAAAEEK(Di...,2,2833.333333,46.251565,1.286647,b,1,14,933.127981


In [25]:
lib['ModifiedPeptideSequence'] = lib['ModifiedPeptideSequence'].apply(lambda x: x.replace('Dimethyl', 'UniMod:199'))

In [78]:
lib.to_csv('/Users/georgwallmann/Documents/data/alphadia_benchmarking/libraries/marvin_scp/MSfragger_library_mod_noLossType_unimod.tsv', sep='\t', index=False)

In [19]:
from alphabase.constants.modification import add_new_modifications

add_new_modifications([
        ('Dimethyl@K','H(4)C(2)'),
        ('Dimethyl@R','H(4)C(2)'),
        ('Dimethyl@Any N-term','H(4)C(2)'),
    ])

In [131]:
target_lib.modification_mapping['Dimethyl@K']

['K(UniMod:36)',
 'K[UniMod:36]',
 'K(UniMod:36)',
 'K[UniMod:36]',
 'K(UniMod:36)',
 'K[UniMod:36]']

In [132]:
# create dense library from diann psm file
target_lib = SWATHLibraryReader()
target_lib.add_modification_mapping(
    {'Oxidation@M':['M(Oxidation)'],
     'Dimethyl@K':['K(Dimethyl)'],
     'Dimethyl@R':['R(Dimethyl)'],
     'Dimethyl@Any N-term':['(Dimethyl)']
    }
)

In [133]:


psm = target_lib.import_file(lib_location)
#target_lib.precursor_df['decoy'] = 0
#target_lib._fragment_mz_df = target_lib._fragment_mz_df[['b_z1', 'b_z2', 'y_z1', 'y_z2']]
#target_lib._fragment_intensity_df = target_lib._fragment_intensity_df[['b_z1', 'b_z2', 'y_z1', 'y_z2']]

#target_lib._precursor_df['elution_group_idx'] = np.arange(len(target_lib._precursor_df))

In [118]:
target_lib._precursor_df = target_lib._psm_df

In [127]:
target_lib.psm_df

,sequence,charge,rt,mobility,mods,mod_sites


In [34]:
target_lib.precursor_df['mods'] = target_lib.precursor_df['mods'].apply(lambda x: x.replace('Dimethyl:2H(4)', 'Dimethyl:2H(4)13C(2)'))

In [9]:
target_lib._precursor_df

,sequence,charge,rt,mobility,mods,mod_sites,nAA,frag_start_idx,frag_stop_idx,rt_norm,precursor_mz,ccs,decoy,elution_group_idx
0,AGAHLQGGAK,3,4.004517,0.747463,Dimethyl:2H(4)@Any N-term;Dimethyl:2H(4)@K,0;10,10,0,9,0.102417,325.205819,455.435969,0,0
1,AISGVHTVR,3,11.848353,0.722217,Dimethyl:2H(4)@Any N-term,0,9,9,17,0.303025,324.536000,440.065864,0,1
2,AMHGELQK,3,8.046609,0.718743,Dimethyl:2H(4)@Any N-term;Dimethyl:2H(4)@K,0;8,8,17,24,0.205794,326.527793,437.911853,0,2
3,EGERPSLR,3,8.817694,0.712443,Dimethyl:2H(4)@Any N-term,0,8,24,31,0.225515,325.855510,434.086124,0,3
4,EVDRPGSK,3,4.305813,0.715224,Dimethyl:2H(4)@Any N-term;Dimethyl:2H(4)@K,0;8,8,31,38,0.110122,317.861829,435.933065,0,4
5,EVVSHVIK,3,14.875842,0.713159,Dimethyl:2H(4)@Any N-term;Dimethyl:2H(4)@K,0;8,8,38,45,0.380453,325.554337,434.527761,0,5
6,GVLFHSGGK,3,19.339818,0.729023,Dimethyl:2H(4)@Any N-term;Dimethyl:2H(4)@K,0;9,9,45,53,0.494621,322.538798,444.251612,0,6
7,HQEHILR,3,6.846526,0.710371,Dimethyl:2H(4)@Any N-term,0,7,53,59,0.175102,322.192345,432.892242,0,7
8,IVHSSMTR,3,6.678781,0.716230,Dimethyl:2H(4)@Any N-term,0,8,59,66,0.170812,321.517841,436.475522,0,8
9,LAGGAQHVR,3,5.802185,0.710272,Dimethyl:2H(4)@Any N-term,0,9,66,74,0.148392,314.192345,432.986819,0,9


In [37]:
target_lib.calc_fragment_mz_df()

In [38]:
target_lib.fragment_mz_df

,b_z1,b_z2,y_z1,y_z2,b_modloss_z1,b_modloss_z2,y_modloss_z1,y_modloss_z2
0,106.107507,53.557392,872.516093,436.761685,0.0,0.0,0.0,0.0
1,163.128971,82.068124,815.494629,408.250953,0.0,0.0,0.0,0.0
2,234.166085,117.586681,744.457516,372.732396,0.0,0.0,0.0,0.0
3,371.224996,186.116136,607.398604,304.202940,0.0,0.0,0.0,0.0
4,484.309060,242.658168,494.314540,247.660908,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...
91,419.271278,210.139277,553.284120,277.145698,0.0,0.0,0.0,0.0
92,516.324042,258.665659,456.231356,228.619316,0.0,0.0,0.0,0.0
93,653.382954,327.195115,319.172444,160.089860,0.0,0.0,0.0,0.0
94,740.414982,370.711129,232.140416,116.573846,0.0,0.0,0.0,0.0


In [12]:
decoy_lib = SpecLibBase()
decoy_lib._precursor_df = target_lib.precursor_df.copy()
decoy_lib._fragment_mz_df = target_lib._fragment_mz_df.copy()
decoy_lib._fragment_intensity_df = target_lib._fragment_intensity_df.copy()

#decoy_lib.import_file(brunner_lib)
decoy_lib.decoy = 'diann'
decoy_lib.append_decoy_sequence()
decoy_lib.calc_precursor_mz()
decoy_lib.precursor_df = decoy_lib.precursor_df[decoy_lib.precursor_df['decoy'] == 1]

In [13]:
from peptdeep.pretrained_models import ModelManager
from alphabase.peptide.fragment import get_charged_frag_types

frag_types = get_charged_frag_types(
    ['b','y'], 
    2
)

model_mgr = ModelManager()

model_mgr.nce = 30
model_mgr.instrument = 'timsTOF'

In [14]:
res = model_mgr.predict_all(
    decoy_lib.precursor_df,
    predict_items=['ms2'],
    frag_types = frag_types,
)

2023-02-15 13:52:09> Using multiprocessing with 8 processes ...
2023-02-15 13:52:09> Predicting ms2 ...


100%|██████████| 46/46 [00:49<00:00,  1.08s/it]


In [15]:
decoy_lib._precursor_df = res['precursor_df']
decoy_lib._fragment_mz_df = res['fragment_mz_df']
decoy_lib._fragment_intensity_df = res['fragment_intensity_df']

In [16]:
decoy_lib._precursor_df[['frag_start_idx']] += len(target_lib._fragment_mz_df)
decoy_lib._precursor_df[['frag_stop_idx']] += len(target_lib._fragment_mz_df)

In [17]:
output_lib = SpecLibBase()

output_lib._precursor_df = pd.concat([target_lib._precursor_df, decoy_lib._precursor_df], join='inner')
output_lib._fragment_mz_df = pd.concat([target_lib._fragment_mz_df, decoy_lib._fragment_mz_df], join='inner')
output_lib._fragment_intensity_df = pd.concat([target_lib._fragment_intensity_df, decoy_lib._fragment_intensity_df], join='inner')

In [18]:
output_lib._precursor_df = output_lib._precursor_df.sort_values(by=['elution_group_idx']).reset_index(drop=True)
output_lib.precursor_df['precursor_idx'] = np.arange(len(output_lib.precursor_df))

In [19]:
from tqdm import tqdm
from alphabase.peptide.precursor import get_mod_seq_formula
from alphabase.constants.isotope import IsotopeDistribution


def calc_isotope_dist(precursor_df, max_isotope = 6, min_intensity = 0.001):
    isotope_dist = IsotopeDistribution()

    col_names = ['i_{}'.format(i) for i in range(max_isotope)]
    precursor_df[col_names] = 0.

    for i in tqdm(range(len(precursor_df))):

        row = precursor_df.iloc[i]
        dist, mono = isotope_dist.calc_formula_distribution(
            get_mod_seq_formula(row['sequence'], row['mods'])
        )
        dist[dist < 0.001] = 0.
        dist = dist / dist.sum()
        precursor_df.loc[i, col_names] = dist[:max_isotope]

In [20]:
calc_isotope_dist(output_lib.precursor_df)

100%|██████████| 465303/465303 [02:52<00:00, 2696.36it/s]


In [21]:
output_lib._precursor_df.drop(columns=['rt'], inplace=True)

In [22]:
output_lib.save_hdf(output_location)

In [27]:
output_lib._precursor_df.iloc[2:5][['sequence', 'charge', 'decoy','mod_seq_charge_hash']]

,sequence,charge,decoy,mod_seq_charge_hash
2,AAAAAAAAAAAAASVGGTIPGPGPGGGQGPGEGGEGQTAR,3,0,2969858240496417433
3,ALAAAAAAAAAAASVGGTIPGPGPGGGQGPGEGGEGQTLR,3,1,2969858240496417433
4,AAAAAAAAAAAAASVGGTIPGPGPGGGQGPGEGGEGQTAR,4,0,2969858240496417434
